In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-2.4.6'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [45.4 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:13 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ge

**This is to remove the previosu jar file downloaded during the wget command the jar file used for the postgres SQL**

In [2]:
!rm "/content/postgresql-42.2.9.jar"
!rm "/content/postgresql-42.2.9.jar.1"
!rm "/content/postgresql-42.2.9.jar.2"

rm: cannot remove '/content/postgresql-42.2.9.jar': No such file or directory
rm: cannot remove '/content/postgresql-42.2.9.jar.1': No such file or directory
rm: cannot remove '/content/postgresql-42.2.9.jar.2': No such file or directory


**This is wget to get the jar file and download in content directory**

In [3]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-02-15 20:18:49--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.30MB/s    in 0.7s    

2021-02-15 20:18:51 (1.30 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



**Py Spark initializing and configuring the postgres jar file for startup**

In [4]:

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

**Loading the file from Amazon website as GZ format and uncompresses it for processing**

In [5]:
# Load in data
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Wireless_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Wireless_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True, timestampFormat="yyyy/MM/dd HH:mm:ss")
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   16414143|R3W4P9UBGNGH1U|B00YL0EKWE|     852431543|LG G4 Case Hard T...|        Wireless|          2|            1|          3|   N|                Y|Looks good, funct...|2 issues  -  Once...| 2015-08-31|
|         US|   50800750|R15V54KBMTQWAY|B00XK95RPQ|     516894650|Selfie Stick Fibl...|        Wireless|          4|    

In [6]:
  df = df.dropna()
  df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   16414143|R3W4P9UBGNGH1U|B00YL0EKWE|     852431543|LG G4 Case Hard T...|        Wireless|          2|            1|          3|   N|                Y|Looks good, funct...|2 issues  -  Once...| 2015-08-31|
|         US|   50800750|R15V54KBMTQWAY|B00XK95RPQ|     516894650|Selfie Stick Fibl...|        Wireless|          4|    

**The Review table updated, data extracted from main data frame for review id, customer id and will be used to update the DB table**

In [8]:
vine_info = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"]).limit(10000)
vine_info.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R3W4P9UBGNGH1U|          2|            1|          3|   N|
|R15V54KBMTQWAY|          4|            0|          0|   N|
| RY8I449HNXSVF|          5|            0|          0|   N|
|R18TLJYCKJFLSR|          5|            0|          0|   N|
|R1NK26SWS53B8Q|          5|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



**The DB initialization, this is Heroku Postgres DB instance, DB created for BigData, there are limitations 10000 records per table**

In [9]:
mode="append"

#rds_connection_string = "postgres:admin@localhost:5432/final_project_stocks"
#jdbc_url = "jdbc:postgresql://<insert endpoint>:5432/my_data_class_db"
jdbc_url = "jdbc:postgresql://ec2-52-3-4-232.compute-1.amazonaws.com:5432/ddv6vu8jpdbjns"
config = {"user":"slcslzlanikhqj",
          "password": "b106eda2173b6ce2c35f34626fb87eafcfd0f52e9ce55f36d776827fef375f71",
          "driver":"org.postgresql.Driver"}
        

**The  Vine review data updated on the main vine in Heroku DB instance**




In [10]:
vine_info.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

**This is for vine review taken full set of data from the data frame for analysis purposes**

In [12]:
vine_info_full_data = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "review_date"])
vine_info_full_data.show()

+--------------+-----------+-------------+-----------+----+-----------+
|     review_id|star_rating|helpful_votes|total_votes|vine|review_date|
+--------------+-----------+-------------+-----------+----+-----------+
|R3W4P9UBGNGH1U|          2|            1|          3|   N| 2015-08-31|
|R15V54KBMTQWAY|          4|            0|          0|   N| 2015-08-31|
| RY8I449HNXSVF|          5|            0|          0|   N| 2015-08-31|
|R18TLJYCKJFLSR|          5|            0|          0|   N| 2015-08-31|
|R1NK26SWS53B8Q|          5|            0|          0|   N| 2015-08-31|
|R11LOHEDYJALTN|          5|            0|          0|   N| 2015-08-31|
|R3ALQVQB2P9LA7|          4|            0|          0|   N| 2015-08-31|
|R3MWLXLNO21PDQ|          5|            0|          0|   N| 2015-08-31|
|R2L15IS24CX0LI|          5|            0|          0|   N| 2015-08-31|
|R1DJ8976WPWVZU|          3|            0|          0|   N| 2015-08-31|
|R3MRWNNR8CBTB7|          5|            0|          0|   N| 2015

**At what date the customer had more review**

In [13]:
from pyspark.sql.functions import desc
date_df = df.groupBy('review_date').agg({"review_date": "count"})
date_df.orderBy(desc("count(review_date)")).show()

+-----------+------------------+
|review_date|count(review_date)|
+-----------+------------------+
| 2015-01-03|             18076|
| 2015-06-03|             17804|
| 2015-08-04|             17554|
| 2015-01-07|             17484|
| 2015-01-05|             17347|
| 2015-08-18|             17027|
| 2015-02-23|             16852|
| 2015-02-18|             16723|
| 2014-12-29|             16327|
| 2015-01-09|             16192|
| 2015-02-24|             16126|
| 2015-02-11|             16091|
| 2015-03-03|             16027|
| 2015-02-19|             15929|
| 2015-01-04|             15867|
| 2015-02-04|             15794|
| 2015-07-22|             15746|
| 2015-02-17|             15697|
| 2015-03-02|             15633|
| 2015-01-12|             15542|
+-----------+------------------+
only showing top 20 rows



**What Product has fewer number of ratings**

In [14]:
product_ratings_df = df.groupby("product_title").agg({"product_title":"count"})\

product_ratings_df.orderBy("count(product_title)").show(truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+
|product_title                                                                                                                                                                                                                                                                                                                                                                                                   |count(product_title)|
+-------------------------------------------------------------------------------------------------------------------------------------------------------

**What is the most review products**

In [15]:
from pyspark.sql.functions import desc
product_ratings_df =df.select(["product_title","star_rating"])\
  .groupby("product_title")\
  .agg({"star_rating": "avg", "product_title":"count"})
product_ratings_df.show(truncate=False)
product_ratings_df.orderBy(desc("count(product_title)")).show(truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------+--------------------+
|product_title                                                                                                                                                                                    |avg(star_rating)  |count(product_title)|
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------+--------------------+
|Evocel Lumia 640 [Dual Layer Series] Hybrid Armor Protector Case For Microsoft Lumia 640 - Retail Packaging, Fire Engine Red (EVO-NK640-SA03)                                                    |4.625             |16                  |
|Audiopure Bluetooth Headphones: Wireless Sport Earbuds 

In [ ]:

i